# RNN Based molucule generation 

Laurent Cetinsoy

In this hands-on we want to generate molecule formulas for denovo-drug discovery. 

For that we need to use Generative models. Generative models are models which goes beyond classification or simple regression : they are able to generate data that look like previously seens dataset. 

There exists a lot of models : 

- Bayesian models like graphical models
- Recurrent models (for sequence generation like texte)
- Variational auto encoders
- Generative adversarial models
- Flow and diffusion models 


In the hands-on we will start by  trainning a character based RNN to generate smile molecules


We want to feed smile representations of molecules to an RNN.
The basic idea is we will train it to predict the next smile token of a molecule given the previous one. 

For instance for the following molecule "CC(=O)NC1=CC=C(O)C=C1" will may give to the model

X = "CC(=O)N" 
y = C

and ask the RNN to learn to predict y given X

Like a standard language model !


## RNN Language model 


A language model is a model which predict the next token of a sequence given the previous ones : 

$ P(X_t | X_{t-1}, X_{t-2}, ..., X_{t-p})  $


This model can be learned with a Recurrent neural network 

$ y = P(X_t | X_{t-1}, X_{t-2}, ..., X_{t-p}) = RNN_{\theta} (X_{t-1}, X_{t-2}, ..., X_{t-p})  $


In order to train such model you need a corpus of data. 



There are two main ways to do that : Word level model or character level model

For character level models, an interesting resource is : http://karpathy.github.io/2015/05/21/rnn-effectiveness/



Explain briefly what is the difference between word based language model and character based language model 

## Loading the data

Dowload the following dataset : https://github.com/joeymach/Leveraging-VAE-to-generate-molecules

In [2]:
import pandas as pd

Import pandas and load the first 1000 lines

In [3]:
df = pd.read_csv('Leveraging-VAE-to-generate-molecules/250k_smiles.csv', nrows=1000)

Display the first rows of the dataframe

In [4]:
df.head(10)

,smiles,logP,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.05060,0.702012,2.084095
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,3.11370,0.928975,3.432004
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778,0.599682,2.470633
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022,0.690944,2.822753
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956,0.789027,4.035182
5,CC[NH+](CC)[C@](C)(CC)[C@H](O)c1cscc1Br\n,2.63740,0.824369,5.091438
6,COc1ccc(C(=O)N(C)[C@@H](C)C/C(N)=N/O)cc1O\n,0.99780,0.327297,2.852316
7,O=C(Nc1nc[nH]n1)c1cccnc1Nc1cccc(F)c1\n,2.33470,0.687612,2.627857
8,Cc1c(/C=N/c2cc(Br)ccn2)c(O)n2c(nc3ccccc32)c1C#N\n,4.28130,0.483079,3.073935
9,C[C@@H]1CN(C(=O)c2cc(Br)cn2C)CC[C@H]1[NH3+]\n,0.88010,0.835024,3.947933


## Processing the data

We need to do the following things : 

- convert smile tokens to numbers
- build  smile token sequences and corresponding labels pairs

Compute the biggest smile molecule size

In [5]:
max_len = df['smiles'].str.len().max()
print(f"Biggest smile molecule size: {max_len}")

Biggest smile molecule size: 106



Code a function **unic_characters(string)** which return the unic characters in a string


In [6]:
def unic_characters(string):
    return sorted(list(set(string)))

Concatenate all smile string of the pandas dataframe and use **unic_characters** to get the unic_characters 

In [8]:
all_smiles = ''.join(df['smiles'])
unique_characters = unic_characters(all_smiles)

Code a function **map_char_to_int(unic_chars)** which returns a dictionnary where each char is assigned an int value. 
Add a character to specify the end of the molecule (like "\n")

> The data already has "\n" at the end of each smile

In [9]:
def map_char_to_int(unic_chars):
    char_to_int = {char: i for i, char in enumerate(unic_chars)}
    return char_to_int

Code a function map_int_to_char(unic_chars) which returns the reverse mapping. 

If you want you can merge both functions in a class

In [10]:
def map_int_to_char(unic_chars):
    int_to_char = {i: char for i, char in enumerate(unic_chars)}
    return int_to_char

For each smile molecule add the ending token to it

In [11]:
char_to_int = map_char_to_int(unique_characters)
int_to_char = map_int_to_char(unique_characters)

## Building the dataset

Now we will create the dataset so that it has the good share for our Keras LSTM model

Remember Keras recurrent models expect a 3D array with shapes (n_examples, seq_len, n_features)



What will be n_features in our case ? 

Code a function **build_X_and_y(string, i_char, seq_lenght)** which takes a string, a **seq_length** number and a position. 


It should create X by by getting all character between i and i + seq_length 
and create y by getting the character following the X sequence
it returns X and y

Test your function on the following string "" with seq_length = 4 and i = [1, 2, 3]

In [14]:
def build_X_and_y(string, i_char, seq_length):
    if i_char + seq_length >= len(string):
        return None, None

    X = string[i_char:i_char + seq_length]
    y = string[i_char + seq_length]

    return X, y

In [18]:
build_X_and_y("", i_char=4, seq_length=1)

(None, None)

In [19]:
build_X_and_y("", i_char=4, seq_length=2)

(None, None)

In [20]:
build_X_and_y("", i_char=4, seq_length=3)

(None, None)

> I think that there is a mistake here and we should replace the string "" by a non empty one?

By using build_X_and_y and map_char_to_int build a list nameed X_train and a list named y_train 

> Let's set the seq_length to 100 (because why not)

In [27]:
import numpy as np
import tensorflow as tf

2025-01-10 00:35:49.287429: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736465749.413544  106416 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736465749.459565  106416 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-10 00:35:49.660768: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [28]:
seq_length = 100
X_train = []
y_train = []

for smile in df['smiles']:
    for i in range(len(smile) - seq_length):
        X, y = build_X_and_y(smile, i, seq_length)

        if X is not None:
            X = [char_to_int[char] for char in X]
            y = char_to_int[y]
            X_train.append(X)
            y_train.append(y)

Create numpy arrays from the lists

In [29]:
X_train = np.array(X_train)
y_train = np.array(y_train)

Reshape the X numpy array (n_examples, seq_lenght, 1)

In [30]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

Normalize X by dividing each values by the total number of unic characters

In [31]:
X_train = X_train / len(char_to_int)
y_train = tf.keras.utils.to_categorical(y_train)

Import Keras and build (at least) a two layered LSTM network with 128 neurone in each.

You can also add Dropoutlayers

Do you think you should use the return_sequences = True ? If yes, when ? 


Add a Dense layer on top with with the appropriate activation function and number of neurones 


In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [33]:
model = Sequential([
    LSTM(128, input_shape=(seq_length, 1), return_sequences=True),
    Dropout(0.2),
    LSTM(128),
    Dropout(0.2),
    Dense(len(char_to_int), activation='softmax')
])

I0000 00:00:1736465774.426240  106416 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2738 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
/home/alex/Desktop/health/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Compile the model with the appropriate loss function and the adam optimizer

In [35]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Train the model on 20 epochs and 10 examples (yeah you read correctly) and check that the model overfits ! 

In [36]:
X_small = X_train[:10]
y_small = y_train[:10]

model.fit(X_small, y_small, epochs=20, batch_size=1)

Epoch 1/20


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(1, 23), output.shape=(1, 33)

If it does not overfit try to fix data prep and model architecture so it does

Create a function **make_prediction(seed_start)** which takes a starting string sequence and uses it to generate a molecule


generate a molecule of your overfitted model

Make a model checkpoint so that the model is saved after each epoch
if you train on a plateform and it stops you do not lose your training 

Now go to your favorite plateform (colab or something else) and train the dataset on the whole data for 10 epochs and batch size 256 

it should take a long time so either follow the class or go take a nap 

Generate between 100 and 1000 molecules. 

create a list where molecules have between 10 and 50 atoms

With rdkit compute the Quantified Estimated Drug likelyness (QED) of each molecule in this subset

Bonus 1 : Using rdkit, compute the quantitative estimation of drug-likeness (QED) of your generated molecules. 

Bonus 2 : try to adapt a transformer model training from hugging face to see if it is better